# Prompting with Prompt Templates for LLM Input/Output with LangChain
* Notebook by Adam Lang
* Date: 6/11/2024
* In this notebook we will go over strategies and best practices for using Prompt Templates in LangChain for improving input/output through the "chain" from the LLM.

## Install libraries

In [1]:
!pip install langchain==0.1.19
!pip install langchain-openai==0.1.6
!pip install langchain-community==0.0.38

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Enter OpenAI key to call LLM

In [2]:
from getpass import getpass

OPENAI_KEY = getpass('Enter your OpenAI API key please: ')

Enter your OpenAI API key please: ··········


## Setup environment variables for OpenAI

In [3]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

# Chat Models and LLMs
* LLMs are of course a core component of LangChain.
* LangChain acts as the interface or hub for LLMs to interact with your application and data. LangChain provides the standard APIs to do so.
* The "LLM Class" is univerally designed to provide a standard API interface for any LLM to interact.

## Commercial LLMs - ChatGPT

In [4]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# Prompt Templates
* A PromptTemplate is actually just a string template we can use to pass variables to in order to generate our final string.
  * Essentially it is a "recipe" for generating language model prompts.
  * These prompt templates can include:
    * 1. instructions
    * 2. few-shot examples
    * 3. specific contexts and questions for particular tasks
* We can then pass these PromptTemplates to any LLM in order to create consistently formatted prompts for the LLM to generate the desired output.
* The great thing about LangChain is that it provides out of the box tools for creating and using prompt templates.
  * These templates are LLM/model agnostic.

## Prompt Template Types
1. PromptTemplate
  * Used for creating **string-based prompts.**
  * Utilizes Python's `str.format` syntax for templating, supporting any number of variables, including scenarios with no variables.

2. ChatPromptTemplate
  * Designed for chat models where the prompt consists of a list of chat messages.
  * Each chat message includes content and a role parameter.
    * For example, in the OpenAI Chat Completions API these roles are:
        * Assistant
        * Human
        * System

3. FewShotChatMessagePromptTemplate
  * Construct a few shot chat template from a set of examples.

# Prompt Template
* Template for a string based prompt.
* By default, uses Python's built in function `str.format` syntax for templating.
* Create custom prompt templates that format the prompt any way you like.

In [5]:
# import PromptTemplate
from langchain.prompts import PromptTemplate

# very simple prompt example
prompt = """Explain to me what is a subarrachnoid hemorrhage in 3 bullet points?"""
prompt_template = PromptTemplate.from_template(prompt)
prompt_template

PromptTemplate(input_variables=[], template='Explain to me what is a subarrachnoid hemorrhage in 3 bullet points?')

In [7]:
# format the prompt
prompt_template.format()

'Explain to me what is a subarrachnoid hemorrhage in 3 bullet points?'

In [8]:
## invoke output response
response = chatgpt.invoke(prompt_template.format())
print(response.content)

- A subarachnoid hemorrhage is a type of stroke caused by bleeding into the space surrounding the brain, known as the subarachnoid space.
- This bleeding is often the result of a ruptured aneurysm, a weakened or bulging blood vessel in the brain that bursts and causes blood to leak into the subarachnoid space.
- Symptoms of a subarachnoid hemorrhage can include sudden and severe headache, nausea, vomiting, neck stiffness, and loss of consciousness, and it is considered a medical emergency that requires immediate treatment.


In [9]:
## more complex prompt with "placeholders"
prompt = """Explain to me briefly about {topic} in {language}."""

prompt_template = PromptTemplate.from_template(prompt)
prompt_template

PromptTemplate(input_variables=['language', 'topic'], template='Explain to me briefly about {topic} in {language}.')

In [13]:
# list of tuples
inputs = [('Generative AI', 'english'),
          ('Artificial Intelligence', 'spanish'),
          ('Deep Learning', 'japanese')]


prompts = [prompt_template.format(topic=topic, language=language) for topic, language in inputs]
prompts

['Explain to me briefly about Generative AI in english.',
 'Explain to me briefly about Artificial Intelligence in spanish.',
 'Explain to me briefly about Deep Learning in japanese.']

In [14]:
## can use python .map() to invoke response
responses = chatgpt.map().invoke(prompts)

In [15]:
# responses
responses

[AIMessage(content='Generative AI is a type of artificial intelligence that is capable of creating new content, such as images, text, or music, that is original and not based on existing data. This technology uses algorithms to generate new content by learning patterns and styles from existing data sets. Generative AI has a wide range of applications, from creating realistic images to composing music, and is often used in creative fields such as art and design.', response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 19, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cabe2309-6573-48af-b563-6f868a272476-0'),
 AIMessage(content='La inteligencia artificial es una rama de la informática que se enfoca en el desarrollo de sistemas y programas capaces de realizar tareas que normalmente requieren de la inteligencia humana, como el aprendizaje, la percepción, el razonamiento y la toma de

In [16]:
## for loop to print each response
for response in responses:
  print(response.content)
  print('-----')

Generative AI is a type of artificial intelligence that is capable of creating new content, such as images, text, or music, that is original and not based on existing data. This technology uses algorithms to generate new content by learning patterns and styles from existing data sets. Generative AI has a wide range of applications, from creating realistic images to composing music, and is often used in creative fields such as art and design.
-----
La inteligencia artificial es una rama de la informática que se enfoca en el desarrollo de sistemas y programas capaces de realizar tareas que normalmente requieren de la inteligencia humana, como el aprendizaje, la percepción, el razonamiento y la toma de decisiones. Estos sistemas utilizan algoritmos y técnicas avanzadas para simular el pensamiento humano y mejorar su desempeño con el tiempo. La inteligencia artificial se aplica en una amplia variedad de campos, como la medicina, la robótica, la industria, la educación y muchos otros, con e

# ChatPromptTemplate
* Standard prompt format to chat models is a list of chat messages.
* Each chat message is associated with content and an additional parameter called `role`.
  * For example, as mentioned above, in OpenAI Chat Completions API a chat message can be associated with an AI assistant, human or system role.

In [17]:
## simple example of ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# simple prompt with placeholders
prompt = """Explain to me briefly about {topic}."""

# instantiate chat_template
chat_template = ChatPromptTemplate.from_template(prompt)
chat_template

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='Explain to me briefly about {topic}.'))])

In [18]:
## make a list of topics to push to the LLM
topics = ['Rocky Mountains','Green Mountains','Swiss Alps']
prompts = [chat_template.format(topic=topic) for topic in topics]
prompts

['Human: Explain to me briefly about Rocky Mountains.',
 'Human: Explain to me briefly about Green Mountains.',
 'Human: Explain to me briefly about Swiss Alps.']

In [19]:
## output response for all 3 prompts
responses = chatgpt.map().invoke(prompts)
for response in responses:
  print(response.content)
  print('-------')

The Rocky Mountains are a major mountain range in western North America, stretching over 3,000 miles from British Columbia in Canada to New Mexico in the United States. They are known for their rugged terrain, diverse wildlife, and stunning natural beauty. The Rockies are a popular destination for outdoor enthusiasts, offering opportunities for hiking, skiing, and wildlife viewing. They also play a crucial role in the region's ecosystem, providing habitat for a wide variety of plant and animal species.
-------
Green Mountains, also known as the Green Mountain Range, is a mountain range in the northeastern United States, primarily located in the state of Vermont. The range is part of the Appalachian Mountains and is known for its lush green forests, scenic beauty, and outdoor recreational opportunities such as hiking, skiing, and camping. The highest peak in the Green Mountains is Mount Mansfield, which reaches an elevation of 4,393 feet. The range is also home to the Long Trail, a popu

In [21]:
## let's view first response index
responses[0]

AIMessage(content="The Rocky Mountains are a major mountain range in western North America, stretching over 3,000 miles from British Columbia in Canada to New Mexico in the United States. They are known for their rugged terrain, diverse wildlife, and stunning natural beauty. The Rockies are a popular destination for outdoor enthusiasts, offering opportunities for hiking, skiing, and wildlife viewing. They also play a crucial role in the region's ecosystem, providing habitat for a wide variety of plant and animal species.", response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 17, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-754c9dc3-20d4-4ebc-b83f-6555660b4f2c-0')

In [23]:
## more complex prompt with a series of messages
messages = [
    ('system', "You are an expert in NFL football history provide brief answers."),
    ('human', 'what is your name?'),
    ('ai', 'my name is AIBot'),
    ('human', '{user_prompt}'),
]

# setup chat_template
chat_template = ChatPromptTemplate.from_messages(messages)
chat_template

ChatPromptTemplate(input_variables=['user_prompt'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert in NFL football history provide brief answers.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='what is your name?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='my name is AIBot')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_prompt'], template='{user_prompt}'))])

In [24]:
# now create text prompts to feed to model
text_prompts = ["what is your name?",
                "explain the Super Bowl to me"]

chat_prompts = [chat_template.format(user_prompt=prompt)
                for prompt in text_prompts]
chat_prompts

['System: You are an expert in NFL football history provide brief answers.\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: You are an expert in NFL football history provide brief answers.\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain the Super Bowl to me']

In [25]:
# look at first prompt
print(chat_prompts[0])

System: You are an expert in NFL football history provide brief answers.
Human: what is your name?
AI: my name is AIBot
Human: what is your name?


In [26]:
## now get the response
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-------')

AI: My name is AIBot
-------
AI: The Super Bowl is the championship game of the National Football League (NFL), played annually between the champions of the league's two conferences, the American Football Conference (AFC) and the National Football Conference (NFC). It is the culmination of the NFL season and is one of the most-watched sporting events in the United States.
-------


### Now modify ChatPromptTemplate
* list of messages different roles

In [28]:
messages = [
    ('system', 'You are an expert in NFL football history, provide detailed answers with examples.'),
    ('human', 'what is your name?'),
    ('ai', "my name is AIBot"),
    ('human', '{user_prompt}'),
]

# setup chat template
chat_template = ChatPromptTemplate.from_messages(messages)
text_prompts = ['what is your name?', 'explain the Ice Bowl to me']
chat_prompts = [chat_template.format(user_prompt=prompt) for prompt in text_prompts]
chat_prompts

['System: You are an expert in NFL football history, provide detailed answers with examples.\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: You are an expert in NFL football history, provide detailed answers with examples.\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain the Ice Bowl to me']

In [29]:
## for loop to invoke responses
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('---------')

AI: My name is AIBot
---------
The Ice Bowl refers to the NFL Championship Game played on December 31, 1967, between the Green Bay Packers and the Dallas Cowboys at Lambeau Field in Green Bay, Wisconsin. The game is famously known for being one of the coldest games in NFL history, with temperatures reaching -13°F (-25°C) and wind chills as low as -48°F (-44°C).

The game was a hard-fought battle between two powerhouse teams, with the Packers ultimately winning 21-17. The most iconic moment of the game came in the final seconds when Packers quarterback Bart Starr scored the game-winning touchdown on a quarterback sneak, despite the freezing conditions and a slippery field covered in ice.

The Ice Bowl is considered one of the greatest games in NFL history and is remembered for the extreme weather conditions, the toughness of the players, and the dramatic finish. It solidified the Packers' legacy as one of the greatest teams in NFL history and remains a legendary moment in football lore.

# PromptTemplate and ChatPromptTemplate BOTH support the LangChain LCEL
* `PromptTemplate` and `ChatPromptTemplate` implement the "runnable interface" which is the basic building block of the LangChain Expression Language (LCEL). This means they support calls via:
  * `invoke`
  * `ainvoke`
  * `stream`
  * `astream`
  * `batch`
  * `abatch`
  * `astream_log`

* `PromptTemplate` accepts a dictionary (of the prompt variables) and returns a `StringPromptValue`.
* A `ChatPromptTemplate` accepts a dictionary and returns a `ChatPromptValue`.

In [32]:
# setup list of prompts
text_prompts = ['what is your name?', 'explain what the west coast offense is']
chat_prompts = [chat_template.invoke({'user_prompt' : prompt}) for prompt in text_prompts]
chat_prompts

[ChatPromptValue(messages=[SystemMessage(content='You are an expert in NFL football history, provide detailed answers with examples.'), HumanMessage(content='what is your name?'), AIMessage(content='my name is AIBot'), HumanMessage(content='what is your name?')]),
 ChatPromptValue(messages=[SystemMessage(content='You are an expert in NFL football history, provide detailed answers with examples.'), HumanMessage(content='what is your name?'), AIMessage(content='my name is AIBot'), HumanMessage(content='explain what the west coast offense is')])]

In [33]:
chat_prompts[1]

ChatPromptValue(messages=[SystemMessage(content='You are an expert in NFL football history, provide detailed answers with examples.'), HumanMessage(content='what is your name?'), AIMessage(content='my name is AIBot'), HumanMessage(content='explain what the west coast offense is')])

In [34]:
# this will output as formatted text
print(chat_prompts[1].to_string())

System: You are an expert in NFL football history, provide detailed answers with examples.
Human: what is your name?
AI: my name is AIBot
Human: explain what the west coast offense is


In [35]:
# list of messages
chat_prompts[1].to_messages()

[SystemMessage(content='You are an expert in NFL football history, provide detailed answers with examples.'),
 HumanMessage(content='what is your name?'),
 AIMessage(content='my name is AIBot'),
 HumanMessage(content='explain what the west coast offense is')]

In [36]:
## get responses from LLM
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('----------')

My name is AIBot
----------
The West Coast Offense is a style of offensive strategy in American football that was popularized by Bill Walsh, the former head coach of the San Francisco 49ers. It is characterized by short, quick passes that are designed to stretch the defense horizontally and create mismatches in space. The offense emphasizes precision timing between the quarterback and receivers, as well as the use of running backs and tight ends as receiving threats.

Key principles of the West Coast Offense include:

1. Short, high-percentage passes: Quarterbacks in the West Coast Offense are trained to make quick reads and get the ball out of their hands rapidly to receivers running short routes. This helps to move the chains and sustain drives.

2. Timing and rhythm: Receivers are expected to run precise routes and be in the right spot at the right time to catch the ball. Quarterbacks must have a strong understanding of the offense and be able to anticipate where their receivers wil

# FewShotChatMessagePromptTemplate
* This is a Chat prompt template that supports few-shot examples.
* The high level structure of produced by this prompt template is a list of messages of prefix message(s), example message(s), and suffix message(s).
* This structure enables creating a conversation with intermediate examples such as:
  * `System: You are a helpful AI assistant`
  * `Human: What is 5 + 5`
  * `AI: 10`
  * `Human: What is 6*8`
  * `AI: 48`
  * `Human: What is 10 - 2`
* The idea is that you give the system a few shot examples and then it has context to hopefully answer your prompts.

#### Define the few-shot examples you would like to include

In [37]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# load up few shot examples
example_data = [
    ("""Wimbledon 1999 Men's final.
    Two-time defending champion Pete Sampras defeated Andre Agassi in the final, 6–3, 6–4, 7–5 to win the gentlemen's singles tennis title""",
     'Sports'),

    ("""US shocks soviets in ice hockey 4-3""",
     'Sports'),

    ("""On January 28, 1986, the NASA shuttle orbiter mission STS-51-L and the tenth flight of Space Shuttle Challenger (OV-99) broke apart 73 seconds into its flight, killing all seven crew members, which consisted of five NASA astronauts and two payload specialists. The spacecraft disintegrated over the Atlantic Ocean, off the coast of Cape Canaveral, Florida, at 11:39 EST (16:39 UTC).""",
     'Space Exploration'),

    ("""Scores killed as terrorist's car bomb blows up block-long Oklahoma City federal building (April 19); Timothy McVeigh, 27, arrested as suspect (April 21); authorities seek second suspect, link right-wing paramilitary groups to bombing (April 22).""",
     'US Events')
]

example_data

[("Wimbledon 1999 Men's final. \n    Two-time defending champion Pete Sampras defeated Andre Agassi in the final, 6–3, 6–4, 7–5 to win the gentlemen's singles tennis title",
  'Sports'),
 ('US shocks soviets in ice hockey 4-3', 'Sports'),
 ('On January 28, 1986, the NASA shuttle orbiter mission STS-51-L and the tenth flight of Space Shuttle Challenger (OV-99) broke apart 73 seconds into its flight, killing all seven crew members, which consisted of five NASA astronauts and two payload specialists. The spacecraft disintegrated over the Atlantic Ocean, off the coast of Cape Canaveral, Florida, at 11:39 EST (16:39 UTC).',
  'Space Exploration'),
 ("Scores killed as terrorist's car bomb blows up block-long Oklahoma City federal building (April 19); Timothy McVeigh, 27, arrested as suspect (April 21); authorities seek second suspect, link right-wing paramilitary groups to bombing (April 22).",
  'US Events')]

In [38]:
## each tuple becomes a dictionary -- formatted
example_data_formatted = [{'input': input, 'output': output}
                          for input, output in example_data]

example_data_formatted

[{'input': "Wimbledon 1999 Men's final. \n    Two-time defending champion Pete Sampras defeated Andre Agassi in the final, 6–3, 6–4, 7–5 to win the gentlemen's singles tennis title",
  'output': 'Sports'},
 {'input': 'US shocks soviets in ice hockey 4-3', 'output': 'Sports'},
 {'input': 'On January 28, 1986, the NASA shuttle orbiter mission STS-51-L and the tenth flight of Space Shuttle Challenger (OV-99) broke apart 73 seconds into its flight, killing all seven crew members, which consisted of five NASA astronauts and two payload specialists. The spacecraft disintegrated over the Atlantic Ocean, off the coast of Cape Canaveral, Florida, at 11:39 EST (16:39 UTC).',
  'output': 'Space Exploration'},
 {'input': "Scores killed as terrorist's car bomb blows up block-long Oklahoma City federal building (April 19); Timothy McVeigh, 27, arrested as suspect (April 21); authorities seek second suspect, link right-wing paramilitary groups to bombing (April 22).",
  'output': 'US Events'}]

#### Assemble few shot examples into few shot prompt template

In [39]:
data_prompt = ChatPromptTemplate.from_messages(
    [
        ('human','{input}'),
        ('ai', '{output}'),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=data_prompt,
    examples=example_data_formatted,
)

few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': "Wimbledon 1999 Men's final. \n    Two-time defending champion Pete Sampras defeated Andre Agassi in the final, 6–3, 6–4, 7–5 to win the gentlemen's singles tennis title", 'output': 'Sports'}, {'input': 'US shocks soviets in ice hockey 4-3', 'output': 'Sports'}, {'input': 'On January 28, 1986, the NASA shuttle orbiter mission STS-51-L and the tenth flight of Space Shuttle Challenger (OV-99) broke apart 73 seconds into its flight, killing all seven crew members, which consisted of five NASA astronauts and two payload specialists. The spacecraft disintegrated over the Atlantic Ocean, off the coast of Cape Canaveral, Florida, at 11:39 EST (16:39 UTC).', 'output': 'Space Exploration'}, {'input': "Scores killed as terrorist's car bomb blows up block-long Oklahoma City federal building (April 19); Timothy McVeigh, 27, arrested as suspect (April 21); authorities seek second suspect, link right-wing paramilitary groups to bombing (April 22).

In [40]:
## actual few shot prompt
print(few_shot_prompt.format())

Human: Wimbledon 1999 Men's final. 
    Two-time defending champion Pete Sampras defeated Andre Agassi in the final, 6–3, 6–4, 7–5 to win the gentlemen's singles tennis title
AI: Sports
Human: US shocks soviets in ice hockey 4-3
AI: Sports
Human: On January 28, 1986, the NASA shuttle orbiter mission STS-51-L and the tenth flight of Space Shuttle Challenger (OV-99) broke apart 73 seconds into its flight, killing all seven crew members, which consisted of five NASA astronauts and two payload specialists. The spacecraft disintegrated over the Atlantic Ocean, off the coast of Cape Canaveral, Florida, at 11:39 EST (16:39 UTC).
AI: Space Exploration
Human: Scores killed as terrorist's car bomb blows up block-long Oklahoma City federal building (April 19); Timothy McVeigh, 27, arrested as suspect (April 21); authorities seek second suspect, link right-wing paramilitary groups to bombing (April 22).
AI: US Events


#### Now assemble final prompt and use with LLM

In [41]:
final_prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Classify every new news article using a similar format as shown below'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)
final_prompt_template

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Classify every new news article using a similar format as shown below')), FewShotChatMessagePromptTemplate(examples=[{'input': "Wimbledon 1999 Men's final. \n    Two-time defending champion Pete Sampras defeated Andre Agassi in the final, 6–3, 6–4, 7–5 to win the gentlemen's singles tennis title", 'output': 'Sports'}, {'input': 'US shocks soviets in ice hockey 4-3', 'output': 'Sports'}, {'input': 'On January 28, 1986, the NASA shuttle orbiter mission STS-51-L and the tenth flight of Space Shuttle Challenger (OV-99) broke apart 73 seconds into its flight, killing all seven crew members, which consisted of five NASA astronauts and two payload specialists. The spacecraft disintegrated over the Atlantic Ocean, off the coast of Cape Canaveral, Florida, at 11:39 EST (16:39 UTC).', 'output': 'Space Exploration'}, {'input': "Scores killed as terrorist's car bo

### Now we feed the LLM new news articles and ask it to classify based on the few shot prompts

In [42]:
docs = ["""The Los Angeles Lakers offered Dan Hurley what would have seemed like a basketball coach’s dream: a chance to coach one of the world’s most popular franchises, possibly an opportunity to work with LeBron James and a salary that would have doubled his current one.""",
        """Nasa says its Perseverance rover has essentially completed the job it was asked to do when it landed on Mars in February 2021.

The robot's basic requirement was to survey an ancient crater lake and to collect rocks that would aid the quest to identify evidence for past life.
This primary objective had been accomplished, the mission team told a major conference in San Francisco.
The announcement was made on the 1,000th Martian day of the mission."""]


final_prompts = [final_prompt_template.format(input=doc) for doc in docs]
responses = chatgpt.map().invoke(final_prompts)

In [43]:
# responses
responses[0]

AIMessage(content='AI: Sports', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 309, 'total_tokens': 312}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3f68c48a-19e4-405f-b69c-5763757d1c5c-0')

In [44]:
# for loop output
for response in responses:
  print(response.content)
  print('----------')

AI: Sports
----------
AI: Space Exploration
----------


Summary:
* We can see that the AI classification was correct based off our few shot templates.

# Partial Prompt Templates
* Just as in other methods, it can make sense to "partial" a prompt template -- eg pass in a subset of the required data values as to create a new prompt template which expects only remaining subset of data values.

* You can use the `partial` function to add some input data first and the rest of the input data later.

In [45]:
from datetime import datetime

# function to get current datetime
def get_datetime():
  now = datetime.now()
  return now.strftime("%m/%d/%Y")

In [46]:
prompt_txt = """Tell me a joke about {topic} on the day {date}"""
prompt = ChatPromptTemplate.from_template(prompt_txt)

In [47]:
prompt = prompt.partial(date=get_datetime)
prompt

ChatPromptTemplate(input_variables=['topic'], partial_variables={'date': <function get_datetime at 0x7dcd010c6b00>}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['date', 'topic'], template='Tell me a joke about {topic} on the day {date}'))])

In [48]:
## lets give topics
topics = ['Plumbers', 'Electricians']
final_prompts = [prompt.format(topic=topic) for topic in topics]
final_prompts

['Human: Tell me a joke about Plumbers on the day 06/12/2024',
 'Human: Tell me a joke about Electricians on the day 06/12/2024']

In [49]:
# get response
responses = chatgpt.map().invoke(final_prompts)
for response in responses:
  print(response.content)
  print('-----------')

Why did the plumber break up with his girlfriend? Because he found out she was just draining him of his money!
-----------
Why did the electrician bring a ladder to the job on 06/12/2024? Because he heard the job was electrifying!
-----------
